<a href="https://colab.research.google.com/github/rubuntu/uaa-417-sistemas-de-gestion-de-bases-de-datos-avanzados/blob/main/06_IO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
%%capture
!pip install expectexception
import expectexception

# Importación y exportación de datos

<!-- requisito: datos/muestra.txt -->
<!-- requisito: datos/csv_sample.txt -->
<!-- requisito: datos/bad_csv.csv -->

Hasta ahora solo hemos tratado con datos que hemos creado dentro de Python. Generar datos aleatorios es útil para probar ideas, pero queremos trabajar con datos reales. En la mayoría de los casos, esos datos se almacenarán en un archivo, ya sea localmente en la computadora o en línea. En este cuaderno aprenderemos cómo leer y escribir datos en archivos.

## Python file handles (`open`)

En Python interactuamos con archivos en el disco usando los comandos "abrir" y "cerrar". Hemos incluido un archivo en la carpeta "datos" llamado "sample.txt". Abrámoslo y leamos su contenido.

In [ ]:
f = open('./data/sample.txt', 'r')
data = f.read()
f.close()

print(data)
print(f)

Observe que "abrimos" el archivo y lo asignamos a "f", "leemos" los datos de "f" y luego cerramos "f". ¿Qué es "f"? Se llama **identificador de archivo**. Es un objeto que conecta Python con el archivo que "abrimos". "Leemos" los datos usando esta conexión y luego, una vez que terminamos, "cerramos" la conexión. Es un buen hábito "cerrar" un identificador de archivo una vez que hayamos terminado con él, por lo que generalmente lo haremos automáticamente usando la palabra clave "with" de Python.

In [ ]:
# f is automatically closed
# at the end of the body of the with statement
with open('./data/sample.txt', 'r') as f:
    print(f.read())

print(f)

También podemos leer líneas individuales de un archivo.

In [ ]:
with open('./data/sample.txt', 'r') as f:
    print(f.readline())

In [ ]:
with open('./data/sample.txt', 'r') as f:
    print(f.readlines())

Escribir datos en archivos es muy similar. La principal diferencia es que cuando "abrimos" el archivo, usaremos la bandera "w" en lugar de "r".

In [ ]:
with open('./data/my_data.txt', 'w') as f:
    f.write('This is a new file.')
    f.write('I am practicing writing data to disk.')

with open('./data/my_data.txt', 'r') as f:
    my_data = f.read()

print(my_data)

No importa con qué frecuencia ejecute la celda anterior, se imprime el mismo resultado. Al abrir el archivo con la bandera `'w'` se sobrescribirá el contenido del archivo. Si queremos agregar algo a lo que ya está en el archivo, tenemos que abrir el archivo con la bandera `'a'` (`'a'` significa _append_).

In [ ]:
with open('./data/my_data.txt', 'a') as f:
    f.write('\nAdding a new line to the file.')

with open('./data/my_data.txt', 'r') as f:
    my_data = f.read()

print(my_data)

Siempre debemos tener cuidado al escribir en el disco, porque podríamos sobrescribir o alterar datos por accidente. También es fácil encontrar errores al trabajar con archivos, porque es posible que no sepamos de antemano si el archivo al que intentamos acceder existe, o podemos mezclar los indicadores 'r', 'w' y 'a'. .

In [ ]:
%%expect_exception IOError

# if a file doesn't exist
# we can't open it for reading
# (but we can open it for writing)

with open('./data/not-exist.txt', 'r') as f:
    f.read()

In [ ]:
%%expect_exception IOError

# we can't read a file open for writing

with open('./data/fail.txt', 'w') as f:
    f.read()

In [ ]:
%%expect_exception IOError

# and we can't write to a file open for reading

with open('./data/sample.txt', 'r') as f:
    f.write('This will fail')

¿Podemos prevenir algunos de estos errores? ¿Cómo sabemos qué archivos hay en el disco?

## módulo `os`

Python tiene un módulo para navegar por el sistema de archivos de la computadora llamado "os". Hay muchas herramientas útiles en el módulo `os`, pero hay dos funciones que son más útiles para buscar archivos.

In [ ]:
import os

# list the contents of the current directory
# ('.' refers to the current directory)
os.listdir('.')

El comando `listdir` es la más simple de las dos funciones que cubriremos. Simplemente enumera el contenido de la ruta del directorio que especificamos. Cuando pasamos `'.'` como argumento, `listdir` buscará en el directorio actual. Enumera todos los cuadernos de Jupyter que estamos usando para el curso, así como el subdirectorio "datos". Podríamos descubrir qué hay en el subdirectorio `data` buscando en `'./data'`.

In [ ]:
os.listdir('./data')

¿Qué pasaría si quisiéramos encontrar todos los archivos y subdirectorios debajo de un directorio en algún lugar de nuestra computadora? Con `listdir` solo vemos los archivos y subdirectorios bajo el directorio particular que estamos buscando. No podemos usar `listdir` para buscar automáticamente en subdirectorios. Para esto necesitamos usar `walk`, que "recorre" todos los subdirectorios debajo de nuestro directorio elegido. No cubriremos `walk` en este curso, pero es una de las herramientas muy útiles (junto con el submódulo `os.path`) para trabajar con archivos en Python, especialmente si está trabajando con muchos archivos de datos diferentes. inmediatamente.

## Archivos CSV

Uno de los formatos más simples y comunes para guardar datos es el de valores separados por comas (CSV).

In [ ]:
with open('./data/csv_sample.txt', 'r') as f:
    csv = f.read()

print(csv)

Este formato se utiliza a menudo para representar tablas de datos. Por lo general, un CSV tendrá filas (separadas por caracteres de nueva línea, `'\n'`) y columnas (separadas por comas). Por lo demás, no se diferencian de cualquier otro archivo de texto. Podemos usar el formato especial de un CSV para crear una lista de listas que representen la tabla.

In [ ]:
list_table = []
with open('./data/csv_sample.txt', 'r') as f:
    for line in f.readlines():
        list_table.append(line.strip().split(','))

list_table

Sin embargo, podemos trabajar con datos tabulares mucho más fácilmente en un Pandas DataFrame. Pandas proporciona un método `read_csv` para leer los datos directamente en un DataFrame.

In [ ]:
import pandas as pd

df = pd.read_csv('./data/csv_sample.txt', index_col=0)
df

El método `read_csv` es muy flexible para manejar el formato de diferentes conjuntos de datos. Algunos conjuntos de datos incluirán encabezados de columna, mientras que otros no. Algunos conjuntos de datos incluirán un índice, mientras que otros no. Algunos conjuntos de datos pueden tener valores separados por tabulaciones, punto y coma u otros caracteres en lugar de comas. Hay opciones en el método `read_csv` para tratar con todos estos. Puede leer sobre ellos en la [documentación de Pandas en `read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html). También lo analizaremos más a fondo en el [Cuaderno de Pandas](08_Pandas.ipynb).

In [ ]:
# an example of downloading
# and importing real data using `read_csv`

if 'factbook.csv' not in os.listdir('./data/'):
    !wget -P ./data/ https://perso.telecom-paristech.fr/eagan/class/igr204/data/factbook.csv

countries = pd.read_csv('./data/factbook.csv', delimiter=';', skiprows=[1])
countries.head()

In [ ]:
# we can also use pandas to write CSV
# using the DataFrame's to_csv method

pd.DataFrame({'a': [0, 3, 10], 'b': [True, True, False]}).to_csv('./data/pd_write.csv')

pd.read_csv('./data/pd_write.csv', index_col=0)

A veces, un CSV no será perfecto. Por ejemplo, tal vez diferentes filas tengan diferentes números de comas. Esto dificulta la interpretación del contenido del archivo como una tabla.

In [ ]:
# the 3rd line only has 2 "columns"

!cat ./data/bad_csv.csv

In [ ]:
# what happens if we try to read this
# into a DataFrame using read_csv?

pd.read_csv('./data/bad_csv.csv', index_col = 0)

El método `read_csv` de Pandas hará todo lo posible para construir una tabla a partir de un CSV mal formateado, pero puede cometer errores. Por ejemplo, 54 se interpretó como un nombre en lugar de una edad, porque solo había 2 columnas en esa línea del archivo. Los conjuntos de datos a menudo contienen errores como formato incorrecto, datos faltantes o errores tipográficos.

**Pregunta:** ¿Cómo podríamos arreglar el CSV mal formateado para que funcione con `read_csv`?

##JSON

JSON significa notación de objetos JavaScript. JavaScript es un lenguaje común para crear aplicaciones web y los archivos JSON se utilizan para recopilar y transmitir información entre aplicaciones JavaScript. Como resultado, existe una gran cantidad de datos en Internet en formato de archivo JSON. Por ejemplo, Twitter y Google Maps utilizan JSON.

Un archivo JSON es esencialmente una estructura de datos construida a partir de listas y diccionarios anidados. Hagamos nuestro propio ejemplo y luego examinaremos un ejemplo descargado de Internet.

In [ ]:
book1 = {'title': 'The Prophet',
         'author': 'Khalil Gibran',
         'genre': 'poetry',
         'tags': ['religion', 'spirituality', 'philosophy', 'Lebanon', 'Arabic', 'Middle East'],
         'book_id': '811.19',
         'copies': [{'edition_year': 1996,
                     'checkouts': 486,
                     'borrowed': False},
                    {'edition_year': 1996,
                     'checkouts': 443,
                     'borrowed': False}]
         }

book2 = {'title': 'The Little Prince',
         'author': 'Antoine de Saint-Exupery',
         'genre': 'children',
         'tags': ['fantasy', 'France', 'philosophy', 'illustrated', 'fable'],
         'id': '843.912',
         'copies': [{'edition_year': 1983,
                     'checkouts': 634,
                     'borrowed': True,
                     'due_date': '2017/02/02'},
                    {'edition_year': 2015,
                     'checkouts': 41,
                     'borrowed': False}]
         }

library = [book1, book2]
library

Tenemos dos libros en nuestra "biblioteca". Ambos libros tienen algunas propiedades comunes: título, autor, identificación y etiquetas. Cada libro puede tener varias etiquetas, por lo que almacenamos esos datos como una lista. Además, puede haber varias copias de cada libro y cada copia también tiene información única, como el año en que se imprimió y cuántas veces se sacó prestado. Tenga en cuenta que si un libro está prestado, también tiene una fecha de vencimiento. Es conveniente almacenar la información sobre las copias múltiples como una lista de diccionarios dentro del diccionario sobre el libro, porque cada copia comparte el mismo título, autor, etc.

Esta estructura es típica de los archivos JSON. Tiene la ventaja de reducir la redundancia de datos. Solo almacenamos el autor y el título una vez, aunque haya varias copias del libro. Además, no almacenamos una fecha de vencimiento para las copias que no están prestadas.

Si tuviéramos que poner estos datos en una tabla, tendríamos que duplicar mucha información. Además, dado que solo se ha prestado una copia de nuestra biblioteca, también tenemos una columna con muchos datos faltantes.    

| index |        title        |          author          |    id    |  genre   |                           tags                            | edition_year | checkouts | borrowed |  due_date  |
|:-----:|:-------------------:|:------------------------:|:--------:|:--------:|:--------------------------------------------------------:|:------------:|:---------:|:--------:|:----------:|
|   0   |     The Prophet     |     Khalil Gibran        |  811.19  |  poetry  | religion, spirituality, philosophy, Lebanon, Arabic, Middle East |     1996     |    486    |   False  |    Null    |
|   1   |     The Prophet     |     Khalil Gibran        |  811.19  |  poetry  | religion, spirituality, philosophy, Lebanon, Arabic, Middle East |     1996     |    443    |   False  |    Null    |
|   2   | The Little Prince   | Antoine de Saint-Exupery | 843.912  | children |     fantasy, France, philosophy, illustrated, fable      |     1983     |    634    |   True   | 2017/02/02 |
|   3   | The Little Prince   | Antoine de Saint-Exupery | 843.912  | children |     fantasy, France, philosophy, illustrated, fable      |     2015     |     41    |   False  |    Null    |


Esto es un gran desperdicio. Dado que los archivos JSON están destinados a compartirse rápidamente a través de Internet, es importante que sean pequeños para reducir la cantidad de recursos necesarios para almacenarlos y transmitirlos.

Podemos escribir nuestra `biblioteca` en el disco usando el módulo `json`.

In [ ]:
import json

with open('./data/library.json', 'w') as f:
    json.dump(library, f, indent=2)

In [ ]:
!cat ./data/library.json

In [ ]:
with open('./data/library.json', 'r') as f:
    reloaded_library = json.load(f)

reloaded_library

In [ ]:
# note that if we loaded it in without JSON
# the file would be interpreted as plain text

with open('./data/library.json', 'r') as f:
    library_string = f.read()

# this isn't what we want
library_string

In [ ]:
# Pandas can also read_json
# notice how it constructs the table
# does it represent the data well?

pd.read_json('./data/library.json')

In [ ]:
# and to_json
df.to_json('./data/example_df.json')

!head ./data/example_df.json

Podemos descargar archivos JSON de muchas formas. A veces lo descargaremos manualmente, pero también podemos usar `wget` como hicimos en el ejemplo CSV. A menudo nos conectaremos a la API de un sitio web que responderá utilizando JSON.

El método `read_json` de Panda es capaz de conectarse directamente a una URL (ya sea la dirección de un archivo JSON o una conexión API) y leer el JSON sin guardar el archivo en nuestro ordenador.

In [ ]:
pd.read_json('https://api.github.com/repos/pydata/pandas/issues?per_page=5')

## Archivos comprimidos (Gzip)

Otra forma de ahorrar almacenamiento y recursos de red es mediante el uso de **compresión**. Muchas veces los conjuntos de datos contendrán patrones que pueden usarse para reducir la cantidad de espacio necesario para almacenar la información.

Un ejemplo sencillo es la siguiente lista de números: 10, 10, 10, 2, 3, 3, 3, 3, 3, 50, 50, 1, 1, 50, 10, 10, 10, 10

En lugar de escribir la lista completa de números (18 enteros), podemos representar la misma información con sólo 14 números: (3, 10), (1, 2), (5, 3), (2, 50), ( 2, 1), (1, 50), (4, 10)

Aquí el primer número de cada par es el número de repeticiones y el segundo número del par es el valor real. Hemos reducido con éxito la cantidad de números que necesitamos para representar los mismos datos. La mayoría de las formas de compresión utilizan una idea similar, aunque las implementaciones reales suelen ser más complejas.

En el mundo de la ciencia de datos, la compresión más común es Gzip (que utiliza el [algoritmo deflate](http://www.infinitepartitions.com/art001.html)). Los archivos gzip terminan con la extensión `.gz`.

In [ ]:
!wget -P ./data/ https://archive.org/stream/TheEpicofGilgamesh_201606/eog_djvu.txt

In [ ]:
import gzip

with open('./data/eog_djvu.txt', 'r') as f:
    text = f.read()

with gzip.open('./data/eog_djvu.txt.gz', 'wb') as f:
    f.write(bytes(text, encoding = 'utf-8'))

!ls -lh ./data/eog*

¡Pudimos comprimir el texto de La Epopeya de Gilgamesh a un tercio de su tamaño original! Recuerde que la compresión depende de los patrones de los datos. El lenguaje tiene muchos patrones, pero ¿qué pasaría si mezclamos todas las letras del texto?

In [ ]:
import numpy as np

with gzip.open('./data/eog_djvu_scrambled.txt.gz', 'wb') as f:
    f.write(np.random.permutation(list(text)))

!ls -lh ./data/eog*

La versión codificada sólo se comprimió a dos tercios del tamaño del original. La compresión no funcionará muy bien en datos aleatorios. La compresión tampoco funciona muy bien en datos que ya son pequeños.

In [ ]:
short_text = 'Hello'

with open('./data/short_text.txt', 'w') as f:
    f.write(short_text)

with gzip.open('./data/short_text.txt.gz', 'wb') as f:
    f.write(bytes(short_text, encoding='utf-8'))

!ls -lh ./data/short_text*

¡El archivo comprimido es más grande que el texto sin formato! Esto se debe a que el archivo comprimido incluye un encabezado, que ocupa una pequeña cantidad de espacio adicional. Además, dado que el texto es tan corto, no es posible utilizar patrones para representar el texto de manera más eficiente. Por lo tanto, normalmente reservamos la compresión para archivos grandes.

Es posible que hayas notado que cuando escribimos archivos Gzip, hemos estado usando una bandera `'wb'` en lugar de una bandera simple `'w'`. Esto se debe a que Gzip no es texto sin formato. Al comprimir el archivo escribimos archivos _binarios_. Los archivos no se pueden leer como texto sin formato.

In [ ]:
# we have to uncompress the file
# before we can read it

!cat ./data/short_text.txt.gz

Sólo debemos usar `'w'` para archivos de texto sin formato (que incluyen CSV y JSON). El uso de `'w'` en lugar de `'wb'` para archivos Gzip u otros archivos que no sean texto plano (por ejemplo, imágenes) podría dañar el archivo.

## Serialización (`pickle`)

A menudo querremos guardar nuestro trabajo en Python y volver a él más tarde. Sin embargo, ese trabajo podría ser un modelo de aprendizaje automático o algún otro objeto complejo en Python. ¿Cómo guardamos objetos complejos de Python? Python tiene un módulo para este propósito llamado `pickle`. Podemos usar `pickle` para escribir un archivo binario que contenga toda la información sobre un objeto Python. Luego podemos cargar ese archivo pickle y reconstruir el objeto en Python.

In [ ]:
pickle_example = ['hello', {'a': 23, 'b': True}, (1, 2, 3), [['dogs', 'cats'], None]]

In [ ]:
%%expect_exception TypeError

# we can't save this as text
with open('./data/pickle_example.txt', 'w') as f:
    f.write(pickle_example)

In [ ]:
import pickle

# we can save it as a pickle
with open('./data/pickle_example.pkl', 'wb') as f:
    pickle.dump(pickle_example, f)

with open('./data/pickle_example.pkl', 'rb') as f:
    reloaded_example = pickle.load(f)

reloaded_example

In [ ]:
# the reloaded example is the same as the original

reloaded_example == pickle_example

Pickle es una herramienta importante para los científicos de datos. El procesamiento de datos y el entrenamiento de modelos de aprendizaje automático pueden llevar mucho tiempo y es útil para guardar puntos de control.

Pandas también tiene métodos `to_pickle` y `read_pickle`.

## Formatos de archivo NumPy

NumPy también tiene métodos para guardar y cargar datos. Son fáciles de usar. Puede encontrar estos cuando trabaje con ciertas bibliotecas de aprendizaje automático que requieren que los datos se almacenen en matrices NumPy. Las matrices NumPy también se utilizan a menudo cuando se trabaja con datos de imágenes.

In [ ]:
sample_array = np.random.random((4, 4))
print(sample_array)

In [ ]:
# to save as plain text
np.savetxt('./data/sample_array.txt', sample_array)

In [ ]:
!cat ./data/sample_array.txt

In [ ]:
print(np.loadtxt('./data/sample_array.txt'))

In [ ]:
# to save as compressed binary
np.save('./data/sample_array.npy', sample_array)

In [ ]:
!cat ./data/sample_array.npy

In [ ]:
print(np.load('./data/sample_array.npy'))

## Temas utilizados por no discutidos:
- Comandos BASH (!)
-`wget`
- `str.split()`
- API